In [ ]:
#|hide
#|default_exp core

# core

> fastdaemon protocol and client

Since this module defines the client, it's important that it has minimal dependencies for us to actually benefit from faster startup. The protocol is largely based on [`fastcgi`](https://github.com/fastai/fastcgi) though much less feature-full.

In [ ]:
#|export
import argparse,socket,struct,sys

In [ ]:
#|hide
from nbprocess.showdoc import *

In [ ]:
import time
from io import BytesIO, BufferedReader
from pathlib import Path
from socketserver import UnixStreamServer,StreamRequestHandler

from fastcore.parallel import threaded

## Protocol

In [ ]:
#|export
def _S(fmt): return struct.Struct('!'+fmt) # use `struct` "network order"
_long_struct = _S('L')

In [ ]:
#|export
def _recv_struct(r, fmt):
    if not isinstance(fmt,struct.Struct): fmt = _S(fmt)
    res = fmt.unpack(r(fmt.size))
    return res[0] if len(res)==1 else res

In [ ]:
#|export
def _str_struct(s): return _long_struct.pack(len(s)) + (s.encode('utf-8') if isinstance(s,str) else s)

In [ ]:
#|export
def send_record(w, c):
    "Send a sequence of length-prefixed utf-8-encoded strings"
    w(b''.join(_str_struct(s) for s in c))

In [ ]:
ss = ['from fastcore.all import *', 'nbprocess_clean --stdin']
f = BytesIO()
send_record(f.write, ss)
b = f.getvalue(); b

b'\x00\x00\x00\x1afrom fastcore.all import *\x00\x00\x00\x17nbprocess_clean --stdin'

In [ ]:
#|export
def _recv_len(r): return _recv_struct(r, _long_struct)

In [ ]:
_recv_len(BufferedReader(BytesIO(b)).read)

26

In [ ]:
#|export
def _recv_string(r):
    l = _recv_len(r)
    return struct.unpack(f'{l}s', r(l))[0].decode('utf-8')

In [ ]:
#|export
def recv_record(r):
    "Receive two variable-length utf-8-encoded strings"
    return [_recv_string(r) for _ in range(2)]

In [ ]:
recv_record(BytesIO(b).read)

['from fastcore.all import *', 'nbprocess_clean --stdin']

In [ ]:
#|export
def _socket_det(port,host,dgram):
    # Source: https://github.com/fastai/fastcore/blob/da9ca219c86190d22f4dbf2c3cad619c477d64a4/fastcore/net.py#L222-L225
    if isinstance(port,int): family,addr = socket.AF_INET,(host or socket.gethostname(),port) # TODO: default to localhost?
    else: family,addr = socket.AF_UNIX,port
    return family,addr,(socket.SOCK_STREAM,socket.SOCK_DGRAM)[dgram]

In [ ]:
#|export
def start_client(port, host=None, dgram=False):
    "Create a `socket` client on `port`, with optional `host`, of type `dgram`"
    # Source: https://github.com/fastai/fastcore/blob/da9ca219c86190d22f4dbf2c3cad619c477d64a4/fastcore/net.py#L242-L247
    family,addr,typ = _socket_det(port,host,dgram)
    s = socket.socket(family, typ)
    s.connect(addr)
    return s

In [ ]:
#|export
def transfer(data, port, host=None, dgram=False):
    "Send a request and receive a reply in one socket using the fastdaemon protocol"
    with start_client(port, host, dgram) as client:
        with client.makefile('wb') as f: send_record(f.write, data)
        with client.makefile('rb') as f: return recv_record(f.read)

In [ ]:
class EchoHandler(StreamRequestHandler):
    def handle(self): self.wfile.write(self.rfile.readline())

In [ ]:
p = Path('fdaemon.sock')
if p.exists(): p.unlink()

@threaded
def _f():
    with UnixStreamServer(str(p), EchoHandler) as srv: srv.handle_request()
_f()
time.sleep(0.2) # wait for server to start

In [ ]:
transfer(['Input via stdin', 'world\n'], str(p)) # trailing \n is required since TestHandler uses `readline`

['Input via stdin', 'world\n']

## Client

In [ ]:
#|export
def _fastdaemon_client(port, host, dgram, args):
    args = ' '.join(args)
    stdin = sys.stdin.read() if not sys.stdin.isatty() else ''
    stdout,stderr = transfer((stdin,args), port, host)
    sys.stderr.write(stderr)
    sys.stdout.write(stdout)

In [ ]:
#|export
def fastdaemon_client(argv=None):
    "Forward `sys.argv` and `sys.stdin` to server and write response to `sys.stdout` and `sys.stderr`"
    if argv is None: argv = sys.argv[1:]
    p = argparse.ArgumentParser(description=fastdaemon_client.__doc__)
    p.add_argument('port', type=str, help='Server port. Use int for TCP, and str for Unix socket')
    p.add_argument('--host', type=str, help='Server host (default: `socket.gethostname()`)', default=None)
    p.add_argument('--dgram', action='store_true', help='Use `SOCK_DGRAM`?', default=None)
    args,rest = p.parse_known_args(argv)
    try: args.port = int(args.port)
    except ValueError: pass
    args.args = rest
    _fastdaemon_client(**vars(args))

In [ ]:
!fastdaemon_client -h

usage: fastdaemon_client [-h] [--host HOST] [--dgram] port

Forward `sys.argv` and `sys.stdin` to server and write response to
`sys.stdout` and `sys.stderr`

positional arguments:
  port         Server port. Use int for TCP, and str for Unix socket

optional arguments:
  -h, --help   show this help message and exit
  --host HOST  Server host (default: `socket.gethostname()`)
  --dgram      Use `SOCK_DGRAM`?


## Export -

In [ ]:
#|hide
#|eval: false
from nbprocess.doclinks import nbprocess_export
nbprocess_export()